In [16]:
!wget https://huggingface.co/monster3412/Text_Sentiment_Analysis/resolve/main/model.h5
!wget https://huggingface.co/monster3412/Text_Sentiment_Analysis/resolve/main/params.json
!wget https://huggingface.co/monster3412/Text_Sentiment_Analysis/resolve/main/tokenizer.pkl
!wget https://huggingface.co/monster3412/Text_Sentiment_Analysis/resolve/main/requirements.txt

--2025-05-28 13:15:58--  https://huggingface.co/monster3412/Text_Sentiment_Analysis/resolve/main/model.h5
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.74, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/cb/95/cb9527b6ca43fbb6ec110fe89c828534aebc8ecdbca5daf4e4d8ca06184c8279/c1d47a03b3381d9f16aa30ec5cfe83b8a68b0cc21117c6bc88901f5a7dcc739d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.h5%3B+filename%3D%22model.h5%22%3B&Expires=1748441758&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0ODQ0MTc1OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2NiLzk1L2NiOTUyN2I2Y2E0M2ZiYjZlYzExMGZlODljODI4NTM0YWViYzhlY2RiY2E1ZGFmNGU0ZDhjYTA2MTg0YzgyNzkvYzFkNDdhMDNiMzM4MWQ5ZjE2YWEzMGVjNWNmZTgzYjhhNjhiMGNjMjExMTdjNmJjODg5MDFmNWE3ZGNjNzM5ZD9yZXNwb25zZS1jb250ZW50LWRpc3

In [17]:
pip install -r requirements.txt

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from tensorflow import keras
import pickle
from nltk.corpus import stopwords
import json
import re
import string
nltk.download('stopwords')

def predict_sentiment(text, link_to_model_address, link_to_tokenizer_address, link_to_preprocessing_params_address):

    model = keras.models.load_model(link_to_model_address)

    # Загрузка токенизатора
    with open(link_to_tokenizer_address, 'rb') as handle:
      tokenizer = pickle.load(handle)

    # Загрузка параметров предобработки
    with open(link_to_preprocessing_params_address, 'r') as f:
      preprocessing_params = json.load(f)

    def remove1(text):
      p=re.compile("<.*?>")
      return p.sub(r"",text)

    text = remove1(text)

    def remove_punctuation(text):
      translator=str.maketrans("","",string.punctuation)
      return text.translate(translator)

    text = remove_punctuation(text)
    stop_words = set(stopwords.words('english'))

    def remove_text(text):
      return " ".join([word for word in text.split() if word.lower() not in stop_words])

    text = remove_text(text)



    # Токенизация и паддинг
    x_seq = tokenizer.texts_to_sequences([text])
    x_pad = pad_sequences(x_seq,
                         maxlen=preprocessing_params['max_length'],
                         padding=preprocessing_params['padding'])

    # Прогноз
    prediction = model.predict(x_pad)

    predictions = model.predict(x_pad)

    sentiment = "positive" if predictions[0][1] > 0.5 else "negative"

    print(f"Predicted sentiment: {sentiment} (Confidence: {max(predictions[0])})")


# Пример использования, здесь необходимо написать свой путь до файлов

text = "This movie was absolutely  bad!"

link_to_model_address='/content/model.h5'

link_to_tokenizer_address='/content/tokenizer.pkl'

link_to_preprocessing_params_address= '/content/params.json'

predict_sentiment(text,link_to_model_address,link_to_tokenizer_address,link_to_preprocessing_params_address)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted sentiment: negative (Confidence: 0.7737241983413696)
